In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('C:\\Users\\Aruna\\Documents\\ACMS-IID\\Code\\Data Cleaning\\Data1.csv')
df.head() 

,label,description
0,Amazon S3,"['public', 'access', 'set', 'bucket', 'policy'..."
1,Amazon S3,"['figure', 'display', 'website', 'domain', 'Er..."
2,Amazon S3,"['test', 'ignore']"
3,Amazon S3,"['account', 'suspended', 'without', 'Notificat..."
4,Amazon S3,"['Unable', 'read', '1000', 'objects', 'bucket'..."


In [2]:
df['label'].value_counts()

Amazon S3                   53174
Amazon EC2                  46535
Amazon RDS                  29525
Amazon CloudFront           22314
Amazon Elastic Beanstalk    22246
Amazon VPC                  21809
Amazon Lambda                8439
Amazon SNS                   4530
Name: label, dtype: int64

In [3]:
tf = pd.read_csv('C:\\Users\\Aruna\\Documents\\ACMS-IID\\Code\\Data Cleaning\\TestData.csv')
tf.head()

,label,description
0,Amazon EC2,"['spot', 'request', 'terminated', 'says', 'ini..."
1,Amazon EC2,"['bump', 'needs', 'looked']"
2,Amazon EC2,['bump']
3,Amazon EC2,"['insufficient', 'data', 'instance', 'status',..."
4,Amazon EC2,"['lamba', 'kms', 'issues', 'encountering', 'de..."


In [4]:
tf['label'].value_counts()

Amazon S3                   100
Amazon SNS                   97
Amazon Lambda                87
Amazon RDS                   71
Amazon Elastic Beanstalk     55
Amazon EC2                   49
Amazon CloudFront            38
Amazon VPC                   34
Name: label, dtype: int64

In [5]:
# from sklearn.model_selection import train_test_split

X_train = df['description']  # this time we want to look at the text
y_train = df['label']

X_test = tf['description']
y_test = tf['label']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(208572, 132702)

In [7]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [8]:
from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('rfc',RandomForestClassifier()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

C:\Users\Aruna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [9]:
predictions = text_clf.predict(X_test)

In [10]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[28  1  1  0  0  8  0  0]
 [ 1 40  1  1  2  3  0  1]
 [ 0 12 28  0  4  9  0  2]
 [ 1  8  0 67  1 10  0  0]
 [ 0 12  1  2 50  4  0  2]
 [ 1  5  1  0  1 92  0  0]
 [ 0  0  1  0  1  4 89  2]
 [ 1  4  1  0  0  1  0 27]]


In [11]:
print(metrics.classification_report(y_test,predictions))

                          precision    recall  f1-score   support

       Amazon CloudFront       0.88      0.74      0.80        38
              Amazon EC2       0.49      0.82      0.61        49
Amazon Elastic Beanstalk       0.82      0.51      0.63        55
           Amazon Lambda       0.96      0.77      0.85        87
              Amazon RDS       0.85      0.70      0.77        71
               Amazon S3       0.70      0.92      0.80       100
              Amazon SNS       1.00      0.92      0.96        97
              Amazon VPC       0.79      0.79      0.79        34

               micro avg       0.79      0.79      0.79       531
               macro avg       0.81      0.77      0.78       531
            weighted avg       0.83      0.79      0.80       531



In [12]:
print(metrics.accuracy_score(y_test,predictions))

0.7928436911487758
